In [1]:
!pip cache purge
!pip install numpy==2.0.0
!pip install scipy==1.10.0

!pip install --upgrade thinc spacy gensim ginza ja_ginza


Files removed: 48
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 59.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 12.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 18.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 

In [2]:
!pip install -U pydantic


In [16]:
!pip show numpy scipy thinc spacy gensim


Name: numpy
Version: 2.0.2
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2024, NumPy Developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

    * Redistributions of source code must retain the above copyright
       notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
       copyright notice, this list of conditions and the following
       disclaimer in the documentation and/or other materials provided
       with the distribution.

    * Neither the name of the NumPy Developers nor the names of any
       contributors may be used to endorse or promote products derived
       from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRI

In [11]:
#q2-1
import pandas as pd

# エクセルファイルを指定して読み込む
file_path = '../data/practice2-2/sisyou_db_h29_01.xlsx'  # ファイル名を適切に変更

# ファイルを読み込む
df = pd.read_excel(file_path)

# '災害状況' 列からNaNを除外
disaster = df['災害状況'].dropna()

# disasterをデータフレームに変換
disaster_df = disaster.to_frame()

# 結果を表示
print("先頭5件:")
print(disaster_df.head())
print("\n末尾5件:")
print(disaster_df.tail())


先頭5件:
                                                災害状況
1  工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前...
2  倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って...
3  会社構内にて車輌の洗車中、足を滑らせ転倒した際に左手をつき、翌朝に左肩の痛みが大きくなり、左...
4                  厩舎2階でバッカン受け入れ作業中、バッカンを落とす穴から落下した。
5  勤務先の食堂施設内で、ダンボールを束ねてビニールの荷造り紐で縛り結んだ時、手が滑り勢いよく壁...

末尾5件:
                                                   災害状況
2691  重機の整備中、待機している台船へ乗船時に、つまずいて高さ1m40㎝～50㎝の所から転落し、足...
2692  新聞配達中、アパートにて2階と3階の配達を終え、1階に下りる時に誤って最後の段で足を滑らせて...
2693  左手にしびれを感じ、中指にも痛みが出始めたたため検査した結果、手根管症候群と中指ばね指と診断...
2694  塗装場所へ移動する為、5尺の脚立をはしご状態にして、約2.3m程上がった屋上へ上る途中に使用...
2695  入浴介助後、利用者（男性48㎏・全介助・車いす）を洗い場から車いすに移動させる際、新人職員が...


In [3]:
#q2-2
import pandas as pd
import spacy
from collections import Counter
import numpy as np

file_path = '../data/practice2-2/sisyou_db_h29_01.xlsx' 
df = pd.read_excel(file_path)
disaster = df['災害状況'].dropna()

nlp = spacy.load('ja_ginza')

# 分かち書き（トークン化）を行う関数
def tokenize(text):
    if isinstance(text, str):  # テキストが文字列の場合のみ処理
        doc = nlp(text)  # nlpを使用して分かち書き
        return [token.text for token in doc if not token.is_stop and not token.is_punct]
    else:
        return []  # 文字列でない場合は空リストを返す

# '災害状況' 列の各テキストをトークン化
disaster_tokens = disaster.apply(tokenize)

# 単語の出現回数をカウント
word_count = Counter([word for tokens in disaster_tokens for word in tokens])

# 出現が20文書に満たない単語を抽出
min_doc_freq = 20
doc_freq = {}
for tokens in disaster_tokens:
    unique_tokens = set(tokens)
    for token in unique_tokens:
        if token not in doc_freq:
            doc_freq[token] = 1
        else:
            doc_freq[token] += 1

# 出現が20文書未満の単語
low_freq_words = [word for word, freq in doc_freq.items() if freq < min_doc_freq]

# 文書数の50%以上で出現する単語を抽出
total_documents = len(disaster_tokens)
min_doc_threshold = np.ceil(total_documents * 0.5)  # 文書数の50%
high_freq_words = [word for word, freq in doc_freq.items() if freq >= min_doc_threshold]

# 除外すべき単語（出現頻度が低すぎるもの、または高すぎるもの）
excluded_words = set(low_freq_words + high_freq_words)

# 除外単語を削除する関数
def remove_excluded_words(tokens):
    return [word for word in tokens if word not in excluded_words]

# 除外された後のトークン
cleaned_disaster_tokens = disaster_tokens.apply(remove_excluded_words)

# 結果を表示
print("出現が20文書に満たない単語:")
print(low_freq_words)
print("50%以上の文書に出現する単語:")
print(high_freq_words)

出現が20文書に満たない単語:
['数', 'コード', '歩', 'よろめい', '進ん', '段位', 'ぐらい', '冷凍', '転げ落ち', '出入', '腓骨', '大きく', '車輌', '剥離', '腱', '洗車', '翌朝', 'バッカン', '落とす', '穴', '受け入れ', '厩舎', '束ね', '勤務先', '荷造り', '縛り', '全治', '紐', 'ヶ月', '切れ', '結ん', 'ビニール', '食堂', '入院', 'ワンボックスカー', 'めまい', '性', '真ん中', '痺れ', '侵入', '箱詰め', '乗せる', '所定', '跨い', 'つこう', '整備', '上ろう', '車体', 'ミキサー', '鉄骨', '組立', '仮', '手摺', '組', 'ラック', '外傷', '立ち上がる', '着け', '棚卸', '片足', 'テナー', '特に', '断裂', 'ずれ落ち', 'ラムダ', '無かっ', 'アキレス', '耐え', '溶け', '半身', '滑らし', '屋上', '立体', 'つく', '歩こう', '耐えよう', '転ば', '身障者', 'ハンド', '道の駅', 'アイスバーン', 'ラッセル', '使い', '日', '帰庫', '出番', '営業車', '背部', '乗務員', 'くらい', 'シーツ', '12', '葉', '束', '開けよう', '桜', '洗う', '分かっ', '右ひじ', '発作', 'パニック', '店長', 'ヒビ', '呼び', '緊急', '雪かき', '左方', '一時停止', '段階', '左前', '飛び出し', '乗り上げ', '後角', '帰局', '集荷', '雪山', '無い', 'エリア', '大きさ', 'かわし', 'グギ', 'ペーパー', '手のひら', '500', '脳', '岸壁', 'ぎっくり腰', 'オビタルサンダー', '震盪', 'っと', '船', 'かける', 'g', '上がり', 'センチ', '離れ', '払っ', '別棟', '捨てる', '積もっ', 'づらかっ', '庫', '戻っ', '更に', '踏ん張っ', '支える', '小走り', '靴底', '付い', '内壁', 

In [4]:
print("単語の出現頻度（上位10件）:")
print(sorted(doc_freq.items(), key=lambda x: x[1], reverse=True)[:10])


単語の出現頻度（上位10件）:
[('際', 1052), ('転倒', 763), ('\n', 707), ('負傷', 610), ('足', 591), ('時', 582), ('作業', 544), ('中', 539), ('骨折', 495), ('内', 458)]


In [12]:
import pandas as pd
import spacy

# spaCyの日本語モデルをロード
nlp = spacy.load('ja_ginza')

# Excelファイルのパスを指定
file_path = '../data/practice2-2/sisyou_db_h29_01.xlsx' 
df = pd.read_excel(file_path)

# '災害状況' 列にNaNがある場合、その行を除外
df_cleaned = df.dropna(subset=['災害状況'])

# '災害状況' 列の最初の文を取得
first_text = df_cleaned['災害状況'].iloc[0]

# テキストの前処理：改行や不要な空白を削除
first_text = first_text.replace("\n", " ").strip()

# 分かち書き（トークン化）を行う関数
def tokenize(text):
    if isinstance(text, str):  # テキストが文字列の場合のみ処理
        doc = nlp(text)  # nlpを使用して分かち書き
        return [token.text for token in doc if not token.is_stop and not token.is_punct]
    else:
        return []  # 文字列でない場合は空リストを返す

# 最初の文の分かち書きを実行
first_text_tokens = tokenize(first_text)

# 結果を表示
print("最初の文の分かち書き結果:")
print(first_text_tokens)


最初の文の分かち書き結果:
['工場', '内', '床', '置い', 'コード', '荷物', '抱え', '状態', '足', '引っ掛かり', 'よろめい', '数', '歩', '前', '進ん', '前方', '作業', '台', '衝突', '受傷']


In [20]:
#q2-3
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# 処理されたテキストデータを結合
cleaned_texts = [" ".join(tokens) for tokens in cleaned_disaster_tokens]

# 単語文書行列の作成
vectorizer = CountVectorizer()
td_matrix = vectorizer.fit_transform(cleaned_texts)

# 単語文書行列をデータフレームに変換
td_matrix_df = pd.DataFrame(
    td_matrix.toarray(), 
    columns=vectorizer.get_feature_names_out()
)

# 結果を確認
print("単語文書行列（先頭5行）:")
print(td_matrix_df.head())


単語文書行列（先頭5行）:
   10  15  20  2人  30  3m  40  50  60  cm  ...  降車  除雪  階段  隙間  頭部  顔面  駐車  \
0   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
1   0   1   0   0   0   0   0   0   0   0  ...   0   0   3   0   0   0   0   
2   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
3   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
4   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   

   駐車場  骨折  高さ  
0    0   0   0  
1    0   1   0  
2    0   0   0  
3    0   0   0  
4    0   0   0  

[5 rows x 462 columns]


In [24]:
#q2-4
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# bi-gram を '_' で作成
bi_grams_tokens = [
    f"{tokens[i]}_{tokens[i+1]}"
    for tokens in cleaned_disaster_tokens
    for i in range(len(tokens) - 1)
]

# 出現回数をカウント
bi_grams_count = Counter(bi_grams_tokens)

# 20回以上出現する bi-gram をフィルタリング
frequent_bi_grams = {bi_gram: count for bi_gram, count in bi_grams_count.items() if count >= 20}

# 各 bi-gram の出現回数をカウントする列を作成
bi_gram_columns = {
    bi_gram: [tokens.count(bi_gram) for tokens in cleaned_disaster_tokens]
    for bi_gram in frequent_bi_grams
}

# bi-gram 列を単語文書行列に一括追加
bi_gram_df = pd.DataFrame(bi_gram_columns)
final_td_matrix = pd.concat([final_td_matrix, bi_gram_df], axis=1)

print("20回以上出現した bi-gram:")
print(frequent_bi_grams)



20回以上出現した bi-gram:
{'工場_内': 165, '作業_台': 44, '滑っ_転倒': 69, '右足_骨折': 22, '足_滑ら': 213, '滑ら_転倒': 105, '転倒_際': 47, '2_階': 88, '左手_小指': 20, 'つまずき_転倒': 40, '転倒_左足': 34, 'バランス_崩し': 223, '崩し_転倒': 41, '2_m': 39, '強打_骨折': 27, '肋骨_骨折': 26, '2_段': 28, '段_目': 53, '駐車場_車': 30, '転倒_\n': 70, '転倒_右手': 27, '凍結_路面': 30, '強打_負傷': 42, '階_1': 20, '1_階': 97, '階_階段': 21, '階段_下り': 26, '足_踏み外し': 53, '当社_工場': 30, 'しまい_負傷': 21, '負傷_\n': 62, '行っ_際': 42, '強打_\n': 26, '\n_際': 103, '1_m': 31, 'トラック_荷台': 65, '荷台_上': 23, '作業_際': 41, '3_段': 33, '高さ_cm': 25, '降りる_際': 29, '移動_時': 35, '際_誤っ': 23, '作業_行っ': 63, '際_右手': 32, '作業_時': 53, '約_m': 22, 'm_下': 21, '終了_後': 68, '右手_中指': 29, '挟み_負傷': 22, '路面_凍結': 61, '転倒_左手': 27, '時_足': 37, '右手首_骨折': 20, '持ち上げ_際': 22, '右手_親指': 29, '向かう_際': 22, '凍結_足': 29, '業務_中': 29, '地面_落下': 26, '内_於い': 24, '\n_痛み': 28, '左手_人差し指': 21, '移動_際': 66, '工事_現場': 65, '転倒_右足': 31, '向かう_途中': 57, '足_滑り': 29, '新聞_配達': 21, '朝刊_配達': 23, '配達_中': 37, '配達_先': 32, '際_路面': 20, '滑り_転倒': 48, 'cm_×': 30, '30_cm': 23, '第_2':

In [5]:
#q2-5
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel

# 単語文書行列をリスト形式で取得
documents = cleaned_disaster_tokens  # Q2-4で得られたトークン化された文書のリスト

# 辞書を作成
dictionary = corpora.Dictionary(documents)

# コーパスを作成（単語文書行列の BoW 表現）
corpus = [dictionary.doc2bow(doc) for doc in documents]

# LDA モデルの作成
model = LdaModel(corpus=corpus, 
                 id2word=dictionary.id2token, 
                 chunksize=2000, 
                 alpha='auto', 
                 eta='auto', 
                 iterations=400, 
                 num_topics=10, 
                 passes=20)

# トピックごとに上位10件の単語を表示
for topic_id, topic in model.show_topics(num_topics=10, num_words=10, formatted=False):
    print(f"トピック {topic_id + 1}:")
    print([word for word, _ in topic])
    print("\n")

ImportError: cannot import name 'triu' from 'scipy.linalg' (/opt/conda/lib/python3.11/site-packages/scipy/linalg/__init__.py)